In [119]:
def mergeCnts(cnts):
    corners = []
    for c in cnts:
        pts = [list(x[0]) for x in c]
        xVals = [x[0] for x in pts]
        yVals = [x[1] for x in pts]
        xMin = min(xVals)
        xMax = max(xVals)
        yMin = min(yVals)
        yMax = max(yVals)

        for i, shape in enumerate(corners):
            left = shape[0][0,0]
            right = shape[1][0,0]
            top = shape[0][0,1]
            bottom = shape[-1][0,1]
            if (xMin >= left and xMin <= right) or (xMax >= left and xMax <= right):
                xMin = min(xMin,left)
                xMax = max(xMax,right)
                yMin = min(yMin,top)
                yMax = max(yMax,bottom)
                del corners[i]

        corners.append(np.array([[[xMin,yMin]],[[xMax,yMin]],[[xMax,yMax]],[[xMin,yMax]]]))
    
    return corners

In [121]:
import cv2
from imutils import contours

inPath = '../../collectedData/sliced/math1/'
file = 'test.png'
outParent = '../../collectedData/sliced/'
outPath = os.path.join(outParent,'.'.join(file.split('.')[:-1]))
print(f'{file} -> {outPath}')
if not os.path.exists(outPath):
    os.mkdir(outPath)
    
image = cv2.imread(os.path.join(inPath,file))
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blur = cv2.GaussianBlur(gray, (9, 9), 0)
thresh = cv2.threshold(blur,0,255,cv2.THRESH_OTSU + cv2.THRESH_BINARY)[1]
invert = cv2.bitwise_not(thresh)
img = invert

# Apply dilate to merge text into meaningful lines/paragraphs.
# Use larger kernel on X axis to merge characters into single line, cancelling out any spaces.
# But use smaller kernel on Y axis to separate between different blocks of text
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 1))
dilate = cv2.dilate(img, kernel, iterations=5)

# Find all contours
contours, hierarchy = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = sorted(contours, key = cv2.contourArea, reverse = True)
cnts = mergeCnts(cnts)

ROI_number = 0
for c in cnts:
    area = cv2.contourArea(c)
    if area > 10:
        x,y,w,h = cv2.boundingRect(c)
        ROI = 255 - image[y:y+h, x:x+w]
        cv2.imwrite('ROI_{}.png'.format(ROI_number), ROI)
        cv2.rectangle(image, (x, y), (x + w, y + h), (random.randint(0,255),random.randint(0,255),random.randint(0,255)), 1)
        ROI_number += 1
cv2.imwrite(os.path.join(outParent,'_markings',str(file)), image)
cv2.imshow('image', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

test.png -> ../../collectedData/sliced/test
